# GS541 phylogenetics homework 1

## Poisson distribution and estimation via maximum likelihood

At the end of this segment, I hope you understand

* why we use the Poisson distribution to model mutation processes
* how model parameter estimation via maximum likelihood "works"

In [ ]:
from scipy.stats import poisson

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [ ]:
def sample_event(probability):
    """Return True with probability `probability`, otherwise return False."""
    return random.random() < probability

Demonstrate that your code is working properly by summing together a large number of outputs from this function and seeing that it's giving (approximately) the expected number of Trues.

In [ ]:
sum(sample_event(0.9) for i in range(1000))

In [ ]:
def count_mutations(generation_count, mutation_probability):
    """Simulate the number of mutations after `generation_count` generations, 
    each of which mutates with probability `mutation_probability`."""
    return sum(sample_event(mutation_probability) for i in range(generation_count))

_Try this function a few times._

Now imagine we have a highly mutable stretch of DNA 10,000 bases long. 
Say this evolves for 700 generations, and each generation each site has a probability of 0.01 of being mutated.

In [ ]:
mutation_counts = pd.Series([count_mutations(700, 0.005) for i in range(10000)])

In [ ]:
mutation_counts.plot.hist(bins=max(mutation_counts))

This distribution looks Poisson-distributed (in fact, [we know that it should be almost exactly Poisson](https://en.wikipedia.org/wiki/Poisson_distribution#Law_of_rare_events)).
So let's model it accordingly. 
First take a moment and read up about the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution).
Then try out various parameters for sampling from the [Poisson distribution as implemented in SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html).

In [ ]:
poisson.rvs(4., size=3)

Try out a couple of values for `pois_parameter` below and see if you can find a value such that the Poisson distribution matches the simulated data.

In [ ]:
ax = mutation_counts.plot.hist(bins=max(mutation_counts))

pois_parameter = 6.
pois_samples = poisson.rvs(pois_parameter, size=10000)
pd.Series(pois_samples).plot.hist(bins=max(pois_samples),ax=ax, alpha=0.5)

Before we continue, take a breath and look what we are doing: we are choosing a parameter value for our _model_ such that when we simulate values from that model, they match (as best as possible) the observed data.

This is the essence of maximum likelihood inference: choosing parameter values for a model such that the model is as likely as possible to generate the observed data.

Now we're going to do such estimation more formally.

The basic insight is that if our model is simple enough we don't have to _simulate_ in order to match the simulated and observed data, but we can just calculate likelihoods directly. 

Let's look at the Probability Mass Function for the Poisson distribution.

In [ ]:
poisson.pmf(4, 2.5)

This says that the probability of getting 4 from a Poission distribution with mean 2.5 is about 0.13.

We can plot the corresponding distribution, which we can think of as the probability of generating 4 mutations from a Poisson process with various means:

In [ ]:
x_values = np.linspace(0., 10.)
probabilities = poisson.pmf(4, x_values)
pd.DataFrame({"x": x_values, "probability": probabilities}).plot(x="x", y="probability")

_Exercise: what is the maximum likelihood value in this plot? Why is that value not surprising?_

Now let's say that we have two sites, and mutations arise in these two sites independently. 
By this independence assumption, [the probability of a particular observation at these two sites is the product of the probabilities at each site](https://en.wikipedia.org/wiki/Independence_(probability_theory)#Two_events).

So, if we have a Poisson distribution with mean 2.5 as before, the probability of getting 4 mutations and 7 mutations is:

In [ ]:
poisson.pmf(4, 2.5) * poisson.pmf(7, 2.5)

_Exercise: make the same plot as just above, but rather than just for one site with 4 mutations, we now have one site with 4 mutations and one site with 7 mutations._

Now we'd like to extend this to many sites, say the 10,000 sites we simulated above. But we encounter a problem!

In [ ]:
np.prod(poisson.pmf(mutation_counts, 3.))

_Exercise: what problem did we encounter?_

We can avoid this problem by taking the log probability of each observation and then summing to get the log likelihood of the whole data set:

In [ ]:
np.sum(poisson.logpmf(mutation_counts, 3.))

In [ ]:
probabilities = np.array([np.sum(poisson.logpmf(mutation_counts, x)) for x in x_values])
pd.DataFrame({"x": x_values, "probability": probabilities}).plot(x="x", y="probability")

_Exercise: what is the maximum likelihood value of the parameter of the Poisson distribution we are using? Why is this not surprising?_

Mutations versus substitutions versus observed mutations.

1. start with a string
2. for every generation, for every site, flip the state
3. note that we aren't getting the right number
3. estimate the number of actual substitutions

In [ ]:
sequence = "0"*1000


1. start with a string
2. for every generation, for every site, sample the number of mutations that will happen, and for each one sample the base that will be sampled
3. estimate the number 